# Script to implement dynamic strawberry shoot growth

In [91]:
from openalea.mtg.algo import orders
from openalea.mtg.traversal import pre_order2
from oawidgets.plantgl import PlantGL

from openalea.strawberry.import_mtgfile import import_mtgfile
from openalea.strawberry import visu2d
import openalea.plantgl.all as pgl

%gui qt

In [92]:
g=import_mtgfile(filename= ["Gariguette"])
g.properties()['order'] = orders(g)


== Line 19: A	A	+	?																																																													
Unknown right symbols <filter object at 0x0000014C26A88610>.
== Line 20: A	A	<	1																																																													
Unknown right symbols <filter object at 0x0000014C26A88400>.
== Line 21: T	T	+	?																																																													
Unknown right symbols <filter object at 0x0000014C26A88820>.
== Line 22: T	T	<	1																																																													
Unknown right symbols <filter object at 0x0000014C26A888B0>.


In [93]:
def plant_ids(g):
    """ the vid of all plants in g

    Args:
        g: an mtg

    Returns: (list of int) the vid of plants in g

    """
    vids = g.component_roots_at_scale(g.root, 1)
    if len(vids) > 0:
        labels = g.property('label')
        return [vid for vid in vids if labels[vid].startswith('P')]
    else:
        return vids
    
plante_ids =plant_ids(g)

In [94]:
scene=visu2d.plot2d(g, vids=[plante_ids[0]],display=False)
PlantGL(scene,group_by_color=False)


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera_animation=[], camer…

# Add dynamic properties on the mtg to simulate developpement
- phyllochrone:  intervening period between the sequential emergence of leaves 

In [95]:
def strawberry_dynamic(g, phyllochron=2):
    """add start_tt and end_tt properties on MTG according to phyllochrone
       The aim is to determine from the phyllochron a number of leaves that have appeared on the final MTG 
       and to calculate a delta that fixes the step of appearance delta_t= phyllochron / number of leaves present 
       in order to include a dynamic of appearance 
       
       Pb: phyllochrone give the number of leaves displays
       
    Parameters
    ----------
    g : Object
        An MTG
    phyllochrone : int or float,
        the intervening period between the sequential emergence of leaves

    Returns
    -------
    Object
        An MTG containing start_tt and end_tt properties
    """
    plants= g.vertices(scale=1) # plantids
    module_scale = 2 #module scale

    for plant in plants: 
        vid = next(g.component_roots_at_scale_iter(plant, scale=module_scale)) #vid of the first module (Trunk)
        time=0 # init a count variable
        #last_time = time + phyllochrone # last time determine the vid of the last components
        
        for module in pre_order2(g, vid):
            time_end = time + phyllochron
            n= g.node(module)
            n.start_tt=time
            n.end_tt=time_end
            n = g.node(module) # node of module
            
            components = [node for node in n.components()] # components of module
            nb_components= len(components) # number of component of the module
            
            last_node = time_end
            
            if nb_components!=0:
                delta_time = phyllochron / last_node # delta time = time interval between components to reach phyllochrone
                for component in components:          
                    component.start_tt = last_node - delta_time # from de last node start of node
                    component.end_tt = last_node # time for the last node
                    last_node -= delta_time
                    print("plant_id :",plant,"module_id: ",module,"node_id :",component, "label :",component.label, "start_tt :",component.start_tt)
        # time +=phyllochron
        
   # return g


strawberry_dynamic(g,phyllochron=2)


plant_id : 1 module_id:  2 node_id : _ProxyNode(3) label : F start_tt : 1.0
plant_id : 1 module_id:  2 node_id : _ProxyNode(9) label : F start_tt : 0.0
plant_id : 1 module_id:  2 node_id : _ProxyNode(14) label : F start_tt : -1.0
plant_id : 1 module_id:  2 node_id : _ProxyNode(20) label : F start_tt : -2.0
plant_id : 1 module_id:  2 node_id : _ProxyNode(26) label : F start_tt : -3.0
plant_id : 1 module_id:  2 node_id : _ProxyNode(32) label : F start_tt : -4.0
plant_id : 1 module_id:  2 node_id : _ProxyNode(37) label : F start_tt : -5.0
plant_id : 1 module_id:  2 node_id : _ProxyNode(42) label : F start_tt : -6.0
plant_id : 1 module_id:  2 node_id : _ProxyNode(45) label : f start_tt : -7.0
plant_id : 1 module_id:  2 node_id : _ProxyNode(50) label : f start_tt : -8.0
plant_id : 1 module_id:  2 node_id : _ProxyNode(55) label : f start_tt : -9.0
plant_id : 1 module_id:  2 node_id : _ProxyNode(63) label : f start_tt : -10.0
plant_id : 1 module_id:  2 node_id : _ProxyNode(72) label : ht star

In [96]:
def dynamic2_strawberry(g,phyllochron):
    """ dd start_tt and end_tt properties on MTG according to phyllochrone
    On all element of MTG at scale 3, calculate a delta_t according to phyllochrone/nb_element
    for each modules. 

    Pb: phyllochron is apply on the whole plant and not on the part of the plant
        
    Parameters
    ----------
    g : object
        An MTG
    phyllochron : int or float
        the intervening period between the sequential emergence of leaves
    
    Returns
    -------
    Object
        An MTG containing start_tt and end_tt properties    
    """
    plants= g.vertices(scale=1)
    module_scale = 2 
    
    for plant in plants:
        vid=next(g.component_roots_at_scale_iter(plant, scale=module_scale))
        tt= 0
        for module in pre_order2(g,vid):
            time_end = tt + phyllochron
            n= g.node(module)
            n.start_tt=tt
            n.end_tt=time_end
            nodes = [node for node in n.components()]
            
            nb_nodes = len(nodes)
            node_tt= tt
            
            if nb_nodes!=0:
                dtt= phyllochron/nb_nodes
                for node in nodes:
                    node.start_tt = node_tt
                    node.end_tt = node_tt + dtt
                    node_tt+=dtt
                    print("plant_id :",plant,"module_id: ",module,"node_id :",node, "label :",node.label, "start_tt :",node.start_tt)

dynamic2_strawberry(g,phyllochron=2)

plant_id : 1 module_id:  2 node_id : _ProxyNode(3) label : F start_tt : 0
plant_id : 1 module_id:  2 node_id : _ProxyNode(9) label : F start_tt : 0.15384615384615385
plant_id : 1 module_id:  2 node_id : _ProxyNode(14) label : F start_tt : 0.3076923076923077
plant_id : 1 module_id:  2 node_id : _ProxyNode(20) label : F start_tt : 0.46153846153846156
plant_id : 1 module_id:  2 node_id : _ProxyNode(26) label : F start_tt : 0.6153846153846154
plant_id : 1 module_id:  2 node_id : _ProxyNode(32) label : F start_tt : 0.7692307692307693
plant_id : 1 module_id:  2 node_id : _ProxyNode(37) label : F start_tt : 0.9230769230769231
plant_id : 1 module_id:  2 node_id : _ProxyNode(42) label : F start_tt : 1.076923076923077
plant_id : 1 module_id:  2 node_id : _ProxyNode(45) label : f start_tt : 1.2307692307692308
plant_id : 1 module_id:  2 node_id : _ProxyNode(50) label : f start_tt : 1.3846153846153846
plant_id : 1 module_id:  2 node_id : _ProxyNode(55) label : f start_tt : 1.5384615384615383
plant_

# Autre test
## Add start_tt property  on leaf element in mtg according to phyllochrone
start_tt was calculated for each leaf if leaf is emerged (F) and module order is visible (ie module order contains at least on leaf emerged)
1. add property if module is visible or not _is_visible = True
2. for each leaf in visble module add start_tt property according to phyllochrone 

1- module is visible or not  

In [98]:
def visible_module(g):
  """add _is_visible property on module scale
  Module is considered like visible when at least one components of the module is F

  Parameters
  ----------
  g : MTG
      an mtg
  
  return
  ------
    add  property is_visible on module scale if module is visible (ie contain one leaf emerged (F) is_visble=True else is_visble==False) 
  """
  modules=[vid for vid in g.vertices_iter(scale=2)]
  _is_visible_={}
  for module in modules:
      if g.label(g.components(module)[0])=='F':
        _is_visible_[module]=True
      else:
        _is_visible_[module]=False

  g.properties()["_is_visible_"]=_is_visible_
  

visible_module(g)

g.property("_is_visible_")


{2: True,
 4: False,
 10: False,
 15: False,
 21: False,
 27: False,
 33: False,
 38: False,
 43: False,
 46: False,
 51: False,
 56: False,
 60: False,
 64: False,
 66: False,
 69: False,
 74: True,
 76: False,
 79: False,
 86: False,
 92: False,
 98: False,
 104: False,
 109: False,
 114: False,
 119: False,
 124: False,
 129: False,
 133: False,
 137: False,
 139: False,
 142: False,
 147: True,
 149: False,
 152: False,
 158: False,
 165: False,
 168: False,
 170: False,
 175: False,
 179: False,
 185: False,
 191: False,
 193: False,
 198: False,
 202: False,
 207: False,
 213: False,
 219: False,
 223: False,
 229: False,
 234: False,
 239: False,
 244: False,
 247: False,
 253: False,
 257: False,
 264: True,
 266: False,
 273: False,
 280: False,
 285: False,
 290: False,
 296: False,
 301: False,
 306: False,
 311: False,
 316: False,
 321: False,
 327: True,
 329: False,
 332: False,
 337: False,
 343: False,
 348: False,
 353: False,
 358: False,
 363: False,
 368: False,
 3

In [106]:

def add_start_tt(g,phyllochrone=2):
    """add start_tt property on leaf element if module is visible (ie. if module contains at least one emerge leaf)
       start_tt is time indicator of leaf which depend of phyllochrone and module. 
       first leaf take initial value of phyllochrone. And each each successive leaf add phyllochrone value depending of start_tt value of the parent

    Parameters
    ----------
    g : MTG
        an mtg
    phyllochrone : int, optional
        is the intervening period between the sequential emergence of leaves, by default 1
    
    return
    ------
        add start_tt property on leaf element of visible module
    """
    start_tt={}
    
    modules= [vid for vid in g.vertices_iter(scale=2)]
    
    for module in modules:
        #if g.property("_is_visible_")[module] == True:
            module_components=g.components(module)
            for vid in module_components:
                pid=g.parent(vid)
                if pid is None:
                    start_tt[vid]=1
                else:
                    start_tt[vid]=start_tt[pid]+phyllochrone
                
                g.properties()["start_tt"]=start_tt     

def add_end_tt(g,phyllochrone=2):
    """add start_tt property on leaf element if module is visible (ie. if module contains at least one emerge leaf)
       start_tt is time indicator of leaf which depend of phyllochrone and module. 
       first leaf take initial value of phyllochrone. And each each successive leaf add phyllochrone value depending of start_tt value of the parent

    Parameters
    ----------
    g : MTG
        an mtg
    phyllochrone : int, optional
        is the intervening period between the sequential emergence of leaves, by default 1
    
    return
    ------
        add start_tt property on leaf element of visible module
    """
    end_tt={}
    modules= [vid for vid in g.vertices_iter(scale=2)]
    for module in modules:
       # if g.property("_is_visible_")[module] == True:
        module_components=g.components(module)
        for vid in module_components:
            pid=g.parent(vid)
            if pid is None:
                end_tt[vid]=1
            else:
                end_tt[vid]=end_tt[pid]+phyllochrone
        # else:
        #     continue       
    g.properties()["end_tt"]=end_tt 


add_start_tt(g)


In [107]:
g.property("start_tt")
    

{3: 1,
 9: 3,
 14: 5,
 20: 7,
 26: 9,
 32: 11,
 37: 13,
 42: 15,
 45: 17,
 50: 19,
 55: 21,
 63: 23,
 72: 25,
 5: 3,
 6: 5,
 7: 7,
 8: 9,
 11: 5,
 12: 7,
 13: 9,
 16: 7,
 17: 9,
 18: 11,
 19: 13,
 22: 9,
 23: 11,
 24: 13,
 25: 15,
 28: 11,
 29: 13,
 30: 15,
 31: 17,
 34: 13,
 35: 15,
 36: 17,
 39: 15,
 40: 17,
 41: 19,
 44: 17,
 47: 19,
 48: 21,
 49: 23,
 52: 21,
 53: 23,
 54: 25,
 57: 23,
 58: 25,
 59: 27,
 62: 29,
 61: 29,
 65: 25,
 68: 27,
 71: 29,
 67: 27,
 70: 29,
 75: 1,
 78: 3,
 85: 5,
 91: 7,
 97: 9,
 103: 11,
 108: 13,
 113: 15,
 118: 17,
 123: 19,
 128: 21,
 136: 23,
 145: 25,
 77: 3,
 80: 5,
 81: 7,
 82: 9,
 83: 11,
 84: 13,
 87: 7,
 88: 9,
 89: 11,
 90: 13,
 93: 9,
 94: 11,
 95: 13,
 96: 15,
 99: 11,
 100: 13,
 101: 15,
 102: 17,
 105: 13,
 106: 15,
 107: 17,
 110: 15,
 111: 17,
 112: 19,
 115: 17,
 116: 19,
 117: 21,
 120: 19,
 121: 21,
 122: 23,
 125: 21,
 126: 23,
 127: 25,
 130: 23,
 131: 25,
 132: 27,
 135: 29,
 134: 29,
 138: 25,
 141: 27,
 144: 29,
 140: 27,
 143: 29